In [ ]:
import os
import random
import shutil

# Set paths
dataset_dir = "C:/Users/IKEH JOHN/Desktop/archive/HandGesture/images/"
train_dir = "C:/Users/IKEH JOHN/Desktop/archive/HandGesture/training/"
test_dir = "C:/Users/IKEH JOHN/Desktop/archive/HandGesture/testing/"

os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

split_ratio = 0.7  # 70% for training, 30% for testing

# Iterate through each class folder
for class_folder in os.listdir(dataset_dir):
    class_path = os.path.join(dataset_dir, class_folder)
    
    # Create class folders in training and testing directories
    os.makedirs(os.path.join(train_dir, class_folder), exist_ok=True)
    os.makedirs(os.path.join(test_dir, class_folder), exist_ok=True)
    
    images = os.listdir(class_path)
    
    random.shuffle(images)
    
    split_index = int(len(images) * split_ratio)
    
    train_images = images[:split_index]
    test_images = images[split_index:]
    
    # Move training images to the training directory
    for image in train_images:
        src = os.path.join(class_path, image)
        dest = os.path.join(train_dir, class_folder, image)
        shutil.copy(src, dest)
    
    # Move testing images to the testing directory
    for image in test_images:
        src = os.path.join(class_path, image)
        dest = os.path.join(test_dir, class_folder, image)
        shutil.copy(src, dest)


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),  
    batch_size=32,
    class_mode='categorical' 
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=1,
    validation_data=test_generator,
    validation_steps=test_generator.samples // test_generator.batch_size
)

test_loss, test_acc = model.evaluate(test_generator)
print("Test accuracy:", test_acc)


In [ ]:
from tensorflow.keras.models import load_model

model.save("hand_gesture_model.keras") # We can save the model use packages like pickle, joblib or even h5.


In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

model = load_model("hand_gesture_model.keras")  # Replace with the path to your trained model file

class_labels = {0: 'call me', 1: 'fingers crossed', 2: 'okay', 3: 'paper', 4: 'peace', 5: 'rock' , 6: 'rock on' , 7: 'scissors' , 8: 'thumbs' , 9: 'up'}

# Function to preprocess image
def preprocess_image(img):
    # Resize image to match input size of the model
    img = cv2.resize(img, (150, 150))
    img = img.astype('float32') / 255.0
    img = np.expand_dims(img, axis=0)
    return img

# Function to predict gesture from image
def predict_gesture(img):
    img = preprocess_image(img)
    prediction = model.predict(img)
    predicted_class_index = np.argmax(prediction)
    predicted_class_label = class_labels[predicted_class_index]
    return predicted_class_label

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    
    #cv2.imshow('Frame', frame)
    gesture = predict_gesture(frame)
    cv2.putText(frame, gesture, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow('Frame with Gesture Prediction', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()
